Use this notebook to upload the initial copy of the pretrained baseline models
- Models should be saved as Tensorflow `SavedModel` format and zipped into a `.tar.gz`, stored in some s3 location that this sagemaker notebook instance has access to
- Assumes that no current endpoint exists/no existing model is being deployed

In [1]:
import os
import tensorflow as tf
import joblib

import boto3
from boto3_utils import *

In [3]:
# Change the following names accordingly
BASELINE_MODEL_S3_PATH = "s3://ngwaf-sagemaker/base_model/kaggle_model_test_serving.tar.gz"
ENDPOINT_NAME = "ngwaf-test-base"
MODEL_NAME = "baseline-colab-11-jul-kaggle-rnn" 

DEPLOYMENT_INSTANCE_COUNT = 1
DEPLOYMENT_INSTANCE_TYPE = 'ml.c5.large'

FRAMEWORK_VERSION = "2.8"

's3://ngwaf-sagemaker/base_model/kaggle_model_test_serving.tar.gz'

In [7]:
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.tensorflow.model import TensorFlowModel

# Sagemaker session
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [10]:
# Check if endpoint is already alive
endpoint_exist = False

endpoints = boto3.client("sagemaker").list_endpoints(NameContains=ENDPOINT_NAME)
for ep in endpoints['Endpoints']:
    if ep['EndpointName'] == ENDPOINT_NAME:
        endpoint_exist = True
        break
endpoint_exist

False

In [11]:
if not endpoint_exist:
    serving_model = TensorFlowModel(
        model_data = BASELINE_MODEL_S3_PATH,
        role=role,
        framework_version=FRAMEWORK_VERSION,
        sagemaker_session=sess,
        name=MODEL_NAME
    )

    predictor = serving_model.deploy(
        initial_instance_count=DEPLOYMENT_INSTANCE_COUNT,
        instance_type=DEPLOYMENT_INSTANCE_TYPE,
        endpoint_name=ENDPOINT_NAME,
    )

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Using already existing model: baseline-colab-11-jul-kaggle-rnn


----!